In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline 
# %matplotlib qt 
import torch.nn.functional
import numpy
import skimage.data
import matplotlib.pyplot as plt

import scipy.ndimage

from itertools import permutations

In [ ]:
# def param2theta2d(param, img_shape):
#     w, h = img_shape[0], img_shape[1]
#     print("w", "h", w, h)
#     # w, h = scipy_shape[0], scipy_shape[1]
#     theta = numpy.zeros((3, 3))
#     theta[2, 2] = 1
#     # wolfram alpha input: {{2/w, 0, -1}, {0, 2/h, -1}, {0, 0, 1}} * {{a, b, c}, {d, e, f}, {0, 0, 1}} * {{2/w, 0, -1}, {0, 2/h, -1}, {0, 0, 1}}^-1
#     theta[0, 0] = param[0, 0]
#     theta[0, 1] = param[0, 1] * h / w
#     theta[0, 2] = param[0, 2] * 2 / w + theta[0, 0] + theta[0, 1] - 1
#     theta[1, 0] = param[1, 0] * w / h
#     theta[1, 1] = param[1, 1]
#     theta[1, 2] = param[1, 2] * 2 / h + theta[1, 0] + theta[1, 1] - 1
#     # return torch.from_numpy(theta[None, ...])
#     return theta

# def param2theta3d(scipy_form, scipy_shape):
#     scipy_form = scipy_form.T
#     t, w, h = scipy_shape[0], scipy_shape[2], scipy_shape[1]
#     t, w, h = scipy_shape[0], scipy_shape[2], scipy_shape[1]
#     # extended 2d case with depth as 't' and additional parameters klmnop
#     # wolfram alpha input: {{2/w, 0, 0, -1}, {0, 2/h, 0, -1}, {0, 0, 2/t, -1}, {0, 0, 0, 1}} * {{a, b, k, c}, {d, e, l, f}, {m, n, o, p}, {0, 0, 0, 1}} * {{2/w, 0, 0, -1}, {0, 2/h, 0, -1},{0, 0, 2/t, -1}, {0, 0, 0, 1}}^-1
#     a = scipy_form[0, 0]
#     b = scipy_form[0, 1]
#     k = scipy_form[0, 2]  # 3d
#     c = scipy_form[0, 3]
#     d = scipy_form[1, 0]
#     e = scipy_form[1, 1]
#     l = scipy_form[1, 2]  # 3d
#     f = scipy_form[1, 3]
# 
#     m = scipy_form[2, 0]  # 3d
#     n = scipy_form[2, 1]  # 3d
#     o = scipy_form[2, 2]  # 3d
#     p = scipy_form[2, 3]  # 3d
# 
#     theta = numpy.array(
#         [
#             [a, b * h / w, k * t / w, a + 2 * c / w + b * h / w + k * t / w - 1],
#             [d * w / h, e, l * t / h, e + l * t / h + d * w / h + 2 * f / h - 1],
#             [m * w / t, h * n / t, o, h * n / t + o + m * w / t + 2 * p / t - 1],
#             # [0, 0, 0, 1],
#         ]
#     )
#     return torch.from_numpy(theta[None, ...])

In [ ]:
img = skimage.data.coins()[50:125, 50:150]
print("img", img.shape)
# plt.imshow(img)
# plt.title("input")
# plt.show()

trf = numpy.eye(3)
trf[0, 2] = 20
trf[1, 2] = 15

# trf[0, 2] = 151
# trf[1, 2] = img.shape[1]

trf[1, 0] = 0.2
trf[0, 1] = 0.3

print('trf', trf)
# print("input", img.shape)
output_shape = [s // 2 for s in img.shape]
scaling = [si / so for si, so in zip(img.shape, output_shape)] + [1.]
print("scaling", scaling)
inv_trf = numpy.linalg.inv(trf)
scipy_form = numpy.array(inv_trf)

# scipy_img = numpy.stack([scipy.ndimage.affine_transform(img[..., i], trf) for i in range(3)], axis=2)
print("scipy_form\n", scipy_form)
scipy_img = scipy.ndimage.affine_transform(img, scipy_form, output_shape=output_shape, order=0)
# print("scipy", scipy_img.shape)
plt.imshow(scipy_img)
plt.title("scipy")
plt.xlabel("x")
plt.show()

# S = numpy.diag(scaling) 
# trf = S.dot(trf)
# # trf[0, 0] *= scaling[0]
# # trf[1, 1] *= scaling[1]
# inv_trf = numpy.linalg.inv(trf)
# scipy_form = numpy.array(inv_trf)
# 
# # scipy_form[0, 0] /= scaling[0]
# scipy_form[1, 1] /= scaling[1]


In [ ]:
def inv_scipy_form2torch_form_2d(inv_scipy_form, ipt_shape, out_shape):
    assert all(zero == 0 for zero in inv_scipy_form[2, :2])
    assert inv_scipy_form[2, 2] == 1
    assert len(ipt_shape) == len(out_shape) == 2
    scaling = [si / so for si, so in zip(ipt_shape, out_shape)] + [1.]
    scaled_scipy = numpy.linalg.inv(numpy.diag(scaling).dot(inv_scipy_form))
    
    transposed_scipy = numpy.eye(3, dtype=scaled_scipy.dtype)
    transposed_scipy[:2, :2] = scaled_scipy[:2, :2].T
    offset = scaled_scipy[:2, 2][::-1]
    transposed_scipy[:2, 2] = offset

    h, w = ipt_shape

    theta = numpy.zeros((2, 3))
    # wolfram alpha input: {{2/w, 0, -1}, {0, 2/h, -1}, {0, 0, 1}} * {{a, b, c}, {d, e, f}, {0, 0, 1}} * {{2/w, 0, -1}, {0, 2/h, -1}, {0, 0, 1}}^-1
    theta[0, 0] = transposed_scipy[0, 0]
    theta[0, 1] = transposed_scipy[0, 1] * h / w
    theta[0, 2] = transposed_scipy[0, 2] * 2 / w + theta[0, 0] + theta[0, 1] - 1
    theta[1, 0] = transposed_scipy[1, 0] * w / h
    theta[1, 1] = transposed_scipy[1, 1]
    theta[1, 2] = transposed_scipy[1, 2] * 2 / h + theta[1, 0] + theta[1, 1] - 1
    return torch.from_numpy(theta[None, ...])

torch_form = inv_scipy_form2torch_form_2d(trf, img.shape, output_shape)
print("torch_form\n", torch_form)

img_nchw = img[None, None, ...].astype("float")
print("nchw", img_nchw.shape)
affine_grid = torch.nn.functional.affine_grid(theta=torch_form, size=[1, 1] + output_shape, align_corners=False)
print("ag", affine_grid.shape)
# affine_grid[..., 0] -= .1967
# affine_grid[..., 1] += .0033
maxs = affine_grid[..., 0].max(), affine_grid[..., 1].max()
# offsets = 
print("maxs", maxs)
print("offsets", [1.0 - ma for ma in maxs])
print('affine_grid', affine_grid.shape, affine_grid.max())
torch_img = torch.nn.functional.grid_sample(torch.from_numpy(img_nchw), affine_grid, align_corners=False, mode="nearest")
print("torch", torch_img.shape, torch_img.max())
plt.imshow(torch_img[0, 0])
plt.title("torch")
plt.xlabel("x")
plt.show()
diff = scipy_img-torch_img[0, 0].numpy()
print(numpy.abs(diff.sum()))
plt.imshow(diff)
plt.title("diff")
plt.colorbar()
plt.show()

In [ ]:
    
    # transposed_scipy[:3, :3] = scipy_form[:3, :3]
    
    # transposed_scipy[1:3, 1:3] = scipy_form[1:3, 1:3].T
    # transposed_scipy[0, 0] = scipy_form[0, 0]
    # transposed_scipy[0, 1] = scipy_form[0, 2]
    # transposed_scipy[0, 2] = scipy_form[0, 1]
    # transposed_scipy[1, 0] = scipy_form[2, 0]
    # transposed_scipy[2, 0] = scipy_form[1, 0]
    
    # transposed_scipy[:2, :2] = scipy_form[1:3, 1:3].T
    # transposed_scipy[2, 2] = scipy_form[2, 3]
    # transposed_scipy[2, 3] = scipy_form[2, 2]
    # transposed_scipy[2, 2] = scipy_form[3, 2]
    # transposed_scipy[3, 2] = scipy_form[2, 2]
    # transposed_scipy[2, 2] = scipy_form[2, 2]
    

    # offset = scipy_form[0, 3], scipy_form[1, 3], scipy_form[2, 3]
    # offset = scipy_form[0, 3], scipy_form[2, 3], scipy_form[1, 3]
    # offset = scipy_form[1, 3], scipy_form[0, 3], scipy_form[2, 3]

    # transposed_scipy[:3, 3] = offset

## 3D

In [ ]:
img = numpy.stack([skimage.data.coins()] * 20, axis=2)
print("img", img.shape)
# plt.imshow(img[0])
# plt.title("input")
# plt.show()

trf = numpy.eye(4)
# trf[0, 0] = 1.15
# trf[0, 1] = .3
# trf[0, 2] = 0.5
# trf[0, 3] = 20
# trf[1, 0] = 0.3
# trf[1, 1] = 1.1
trf[1, 2] = 0.13
trf[1, 3] = 0.06
trf[2, 0] = 0.03
trf[2, 1] = 0.04
trf[2, 2] = 1.1
trf[2, 3] = -20

# print("input", img.shape)
output_shape = [s for s in img.shape]

# scipy_form = numpy.linalg.inv(trf)
scipy_form = numpy.array(trf)

# scipy_img = numpy.stack([scipy.ndimage.affine_transform(img[..., i], trf) for i in range(3)], axis=2)
print("scipy_form\n", scipy_form)
scipy_img = scipy.ndimage.affine_transform(img, scipy_form, output_shape=output_shape, order=0)
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(8, 4), squeeze=False)
ax[0, 0].imshow(scipy_img[150], vmin=0, vmax=255)
ax[0, 0].set_title("scipy")
ax[0, 1].imshow(scipy_img[:, 150], vmin=0, vmax=255)
ax[0, 1].set_title("scipy")
ax[0, 2].imshow(scipy_img[:, :,  10], vmin=0, vmax=255)
ax[0, 2].set_title("scipy")
plt.show()

In [ ]:
def scipy_form2torch_form_3d(scipy_form, img_shape):
    transposed_scipy = numpy.eye(4, dtype=scipy_form.dtype)
    map_pos = {
        (0, 0): (2, 2),
        (0, 1): (2, 1),
        (0, 2): (2, 0),
        (0, 3): (2, 3),
        (1, 0): (1, 2),
        (1, 1): (1, 1),
        (1, 2): (1, 0),
        (1, 3): (1, 3),
        (2, 0): (0, 2),
        (2, 1): (0, 1),
        (2, 2): (0, 0),
        (2, 3): (0, 3),
    }
    for spos, tpos in map_pos.items():
        transposed_scipy[tpos] = scipy_form[spos]
    
    t, h, w = img_shape

    # extended 2d case with depth as 't' and additional parameters klmnop
    # wolfram alpha input: {{2/w, 0, 0, -1}, {0, 2/h, 0, -1}, {0, 0, 2/t, -1}, {0, 0, 0, 1}} * {{a, b, k, c}, {d, e, l, f}, {m, n, o, p}, {0, 0, 0, 1}} * {{2/w, 0, 0, -1}, {0, 2/h, 0, -1},{0, 0, 2/t, -1}, {0, 0, 0, 1}}^-1
    a = transposed_scipy[0, 0]
    b = transposed_scipy[0, 1]
    k = transposed_scipy[0, 2]  # 3d
    c = transposed_scipy[0, 3]
    d = transposed_scipy[1, 0]
    e = transposed_scipy[1, 1]
    l = transposed_scipy[1, 2]  # 3d
    f = transposed_scipy[1, 3]

    m = transposed_scipy[2, 0]  # 3d
    n = transposed_scipy[2, 1]  # 3d
    o = transposed_scipy[2, 2]  # 3d
    p = transposed_scipy[2, 3]  # 3d

    theta = numpy.array(
        [
            [a, b * h / w, k * t / w, a + 2 * c / w + b * h / w + k * t / w - 1],
            [d * w / h, e, l * t / h, e + l * t / h + d * w / h + 2 * f / h - 1],
            [m * w / t, h * n / t, o, h * n / t + o + m * w / t + 2 * p / t - 1],
            # [0, 0, 0, 1],
        ]
    )
    return torch.from_numpy(theta[None, ...])

affine_grid = torch.nn.functional.affine_grid(theta=scipy_form2torch_form_3d(numpy.array(scipy_form), img.shape), size=[1, 1] + output_shape, align_corners=False)
img_ncdhw = img.astype("float")[None, None]
torch_img = torch.nn.functional.grid_sample(torch.from_numpy(img_ncdhw), affine_grid, align_corners=False, mode="bilinear")

diff = numpy.abs(scipy_img - torch_img[0, 0].numpy())
diff_sum = diff.sum()
print("diff", int(diff_sum))
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 8))
ax[1 - 1, 0].imshow(torch_img[0, 0, 150], vmin=0, vmax=255)
ax[1 - 1, 0].set_title("torch")
ax[1 - 1, 1].imshow(torch_img[0, 0, :, 150], vmin=0, vmax=255)
ax[1 - 1, 1].set_title("torch")
ax[1 - 1, 2].imshow(torch_img[0, 0, :, :,  10], vmin=0, vmax=255)
ax[1 - 1, 2].set_title("torch")
ax[2 - 1, 0].imshow(diff[150], vmin=0, vmax=255)
ax[2 - 1, 0].set_title("diff")
ax[2 - 1, 1].imshow(diff[:, 150], vmin=0, vmax=255)
ax[2 - 1, 1].set_title("diff")
ax[2 - 1, 2].imshow(diff[:, :, 10], vmin=0, vmax=255)
ax[2 - 1, 2].set_title("diff")
plt.show()

print('done')

In [ ]:
print("la")

In [ ]:
print(affine_grid[0, ..., 0].max())
plt.imshow(affine_grid[0, ..., 0])
plt.colorbar()

In [ ]:
print(affine_grid[0, ..., 1].max())
plt.imshow(affine_grid[0, ..., 1])
plt.colorbar()



In [ ]:
print(n23_form)





In [ ]:

# # torch_form = numpy.array([[
# #     [ 1.0, -0.5, 0.0], 
# #     [ 0.0,  1.0, 0.0]
# # print("trf\n", trf)
# # print("inv trf\n", inv_trf)
# torch_trf = numpy.array(trf)
# # torch_offset = [0.8317-1.0, 0.19609999999999994]
# # torch_offset = [0.1961, -0.0033]
# # torch_offset = [+0.1967, -0.0033]
# torch_trf[0, 2] = trf[0, 2] / img.shape[0]
# torch_trf[1, 2] = trf[1, 2] / img.shape[1]
# 
# torch_inv_trf = numpy.linalg.inv(torch_trf)
# torch_offset = numpy.eye(3)
# # torch_offset[0, 2] = .2
# torch_offset[1, 2] = -0.2
# # torch_inv_trf = torch_offset.dot(torch_inv_trf)
# torch_inv_trf = torch_inv_trf.dot(torch_offset)
# 
# torch_form = numpy.zeros((2, 3))
# torch_form[:2, :2] = torch_inv_trf[:2, :2].T
# torch_form[:2, 2] = torch_inv_trf[:2, 2][::-1]
# torch_form[0, 2] += torch_offset[0]
# torch_form[1, 2] += torch_offset[1]
# torch_form[:2, 2] = [torch_inv_trf[i, 2] / img.shape[i] for i in range(2)][::-1]
# torch_form[:2, 2] = [torch_inv_trf[i, 2] / img.shape[i] for i in range(2)][::-1]
# torch_form[0, 2] += -0.5
# torch_form[1, 2] += -0.5
# torch_form = numpy.concatenate([torch_inv_trf[:2, :2].T, inv_trf[:2, 2:3][::-1]], axis=1)